# ETL Job

This notebook consists on extract research dataset, performe some transformations on it, and then store structured data in MySQL.

## Extract

### Open file

In [1]:
from zipfile import ZipFile

In [ ]:
research_datasets_file = '../data/research_datasets.zip'

In [2]:
zipped_datasets = ZipFile(research_datasets_file)
zipped_datasets

<zipfile.ZipFile filename='../data/research_datasets.zip' mode='r'>

In [3]:
dataset_files = zipped_datasets.namelist()
dataset_files[:5]

['2020-01-01.csv',
 '2020-01-02.csv',
 '2020-01-03.csv',
 '2020-01-04.csv',
 '2020-01-05.csv']

### Read data

In [5]:
import pandas as pd

In [6]:
def read_research_dataframe(zipfile, filename):
    return pd.read_csv(zipfile.open(filename), sep='|', encoding='ISO-8859-1')

In [7]:
df_source = pd.concat([read_research_dataframe(zipped_datasets, filename) for filename in dataset_files], ignore_index=True)
df_source.sample(5)

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
486,10142726,2020-03-24,Masculino,1987-08-05,gato,moderado,Água,Escrever
1484,10143544,2020-09-15,Masculino,1963-04-16,peixe,moderado,Água,Praticar esporte
818,10142751,2020-05-26,Masculino,1972-08-12,gato,quente,Água,Pintar quadros
907,10142753,2020-06-13,Masculino,1959-12-13,tartaruga,moderado,refrigerante,Dormir
1596,10142385,2020-10-05,Masculino,1968-08-18,cachorro,frio,Água,Assistir TV


In [8]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cod_pessoa        2127 non-null   int64 
 1   data_coleta       2127 non-null   object
 2   genero            2127 non-null   object
 3   data_nascimento   2127 non-null   object
 4   animal_estimacao  2127 non-null   object
 5   clima             2127 non-null   object
 6   bebida_favorita   2127 non-null   object
 7   hobbies           2127 non-null   object
dtypes: int64(1), object(7)
memory usage: 133.1+ KB


## Transform

### Weather's dataset

In [9]:
df_weather = df_source[['clima']].drop_duplicates().reset_index(drop=True)
df_weather = df_weather.reset_index()
df_weather = df_weather.rename(columns={'index': 'weather_id', 'clima': 'weather_description'})
df_weather['weather_id'] += 1
df_weather

,weather_id,weather_description
0,1,frio
1,2,quente
2,3,moderado


### Hobby's dataset

In [10]:
df_hobby = df_source[['hobbies']].drop_duplicates().reset_index(drop=True)
df_hobby = df_hobby.reset_index()
df_hobby = df_hobby.rename(columns={'index': 'hobby_id', 'hobbies': 'hobby_description'})
df_hobby['hobby_id'] += 1
df_hobby

,hobby_id,hobby_description
0,1,Praticar esporte
1,2,Assistir TV
2,3,Ler livros
3,4,Aprender algo novo
4,5,Escrever
5,6,Pintar quadros
6,7,Dormir
7,8,Escutar música


### Drink's dataset

In [11]:
df_drink = df_source[['bebida_favorita']].drop_duplicates().reset_index(drop=True)
df_drink = df_drink.reset_index()
df_drink = df_drink.rename(columns={'index': 'drink_id', 'bebida_favorita': 'drink_description'})
df_drink['drink_id'] += 1
df_drink

,drink_id,drink_description
0,1,Cerveja
1,2,Café
2,3,refrigerante
3,4,Água
4,5,Vinho
5,6,Chá


### Pet's dataset

In [12]:
df_pet = df_source[['animal_estimacao']].drop_duplicates().reset_index(drop=True)
df_pet = df_pet.reset_index()
df_pet = df_pet.rename(columns={'index': 'pet_id', 'animal_estimacao': 'pet_animal'})
df_pet['pet_id'] += 1
df_pet

,pet_id,pet_animal
0,1,peixe
1,2,tartaruga
2,3,gato
3,4,cachorro


### Person's dataset

In [13]:
df_person = df_source[['cod_pessoa', 'genero', 'data_nascimento']].drop_duplicates().reset_index(drop=True)
df_person = df_person.rename(columns={'cod_pessoa': 'person_id', 'genero': 'person_gender', 'data_nascimento': 'person_birthdate'})
df_person

,person_id,person_gender,person_birthdate
0,10141743,Masculino,1981-04-25
1,10141860,Feminino,1966-01-19
2,10142548,Masculino,1973-08-02
3,10142591,Masculino,1997-03-05
4,10142654,Feminino,1969-08-05
...,...,...,...
2122,10142475,Masculino,1955-05-30
2123,10142522,Masculino,1954-12-22
2124,10142881,Masculino,1952-07-30
2125,10143184,Masculino,1984-06-05


### Research's dataset

In [14]:
df_research = df_source \
    .merge(df_weather, left_on='clima', right_on='weather_description') \
    .merge(df_hobby, left_on='hobbies', right_on='hobby_description') \
    .merge(df_drink, left_on='bebida_favorita', right_on='drink_description') \
    .merge(df_pet, left_on='animal_estimacao', right_on='pet_animal') \
    .merge(df_person, left_on='cod_pessoa', right_on='person_id')
df_research = df_research.reset_index()
df_research = df_research.rename(columns={'index': 'research_id', 'data_coleta': 'research_date'})
df_research = df_research[['research_id', 'research_date', 'person_id', 'pet_id', 'drink_id', 'hobby_id', 'weather_id']]
df_research['research_id'] += 1
df_research

,research_id,research_date,person_id,pet_id,drink_id,hobby_id,weather_id
0,1,2020-01-01,10141743,1,1,1,1
1,2,2020-03-17,10141885,1,1,1,1
2,3,2020-03-24,10142225,1,1,1,1
3,4,2020-04-09,10141845,1,1,1,1
4,5,2020-06-08,10143487,1,1,1,1
...,...,...,...,...,...,...,...
2122,2123,2020-11-19,10143582,4,3,8,1
2123,2124,2020-02-04,10141715,4,3,8,2
2124,2125,2020-07-27,10141505,4,3,8,2
2125,2126,2020-09-26,10142547,4,3,8,2


In [15]:
df_research.columns

Index(['research_id', 'research_date', 'person_id', 'pet_id', 'drink_id',
       'hobby_id', 'weather_id'],
      dtype='object')

## Load

### Database connection

In [18]:
from sqlalchemy import create_engine

In [19]:
import os

In [20]:
def get_engine_url_from_env():
    env_vars = ['DB_ADAPTER', 'DB_HOST', 'DB_PORT', 'DB_USERNAME', 'DB_PASSWORD', 'DB_NAME']
    adapter, host, port, username, password, database = [os.environ.get(env_var) for env_var in env_vars]
    return f'{adapter}://{username}:{password}@{host}:{port}/{database}'

In [21]:
engine = create_engine(get_engine_url_from_env())
engine

Engine(mysql+mysqlconnector://user:***@db:3306/research)

### Storing data

In [22]:
with engine.connect() as conn:
    df_weather.to_sql('weather', con=conn, if_exists='append', index=False)
    df_hobby.to_sql('hobby', con=conn, if_exists='append', index=False)
    df_drink.to_sql('drink', con=conn, if_exists='append', index=False)
    df_pet.to_sql('pet', con=conn, if_exists='append', index=False)
    df_person.to_sql('person', con=conn, if_exists='append', index=False)
    df_research.to_sql('research', con=conn, if_exists='append', index=False)